## Using a submodel

In [5]:
!pip install pybamm
import pybamm as pb

In [6]:
dfn = pb.lithium_ion.DFN()
dfn.submodels

{'external circuit': <pybamm.models.submodels.external_circuit.explicit_control_external_circuit.ExplicitCurrentControl at 0x1557163b350>,
 'discharge and throughput variables': <pybamm.models.submodels.external_circuit.discharge_throughput.DischargeThroughput at 0x1557163b380>,
 'porosity': <pybamm.models.submodels.porosity.constant_porosity.Constant at 0x1557163b2f0>,
 'Negative interface utilisation': <pybamm.models.submodels.interface.interface_utilisation.full_utilisation.Full at 0x1557163b440>,
 'Positive interface utilisation': <pybamm.models.submodels.interface.interface_utilisation.full_utilisation.Full at 0x155715c2240>,
 'negative primaryparticle mechanics': <pybamm.models.submodels.particle_mechanics.no_mechanics.NoMechanics at 0x1557163b5f0>,
 'positive primaryparticle mechanics': <pybamm.models.submodels.particle_mechanics.no_mechanics.NoMechanics at 0x1557163b620>,
 'negative primary active material': <pybamm.models.submodels.active_material.constant_active_material.Cons

## Thermal model
Accounts for change in temperature caused by operation of the battery. Avaialble models include:
- Isothermal: constant temp
- Lumped: homogeneous within the battery
- X-lumped: homogeneous across thickness, varies in direction parallel to the current collectors. Requires current collector model
- X-full: varies across the cell

In [7]:
'''
Compare the results of the DFN model with isothermal and x-full. 
Plot the relevant variables, and recall that you can find the list of variables to plot by typing model.variable_names(). 
What differences do you observe in the models?
'''

thermal_options = ["isothermal", "lumped", "x-full"]
sols = []

for option in thermal_options:
    model = pb.lithium_ion.DFN(name=option, options={"thermal": option})    
    sim = pb.Simulation(model)
    sols.append(sim.solve([0, 3600]))

c:\ProgramData\anaconda3\Lib\site-packages\pybamm\simulation.py:120: UserWarning: The default solver changed to IDAKLUSolver after the v25.4.0. release. You can swap back to the previous default by using `pybamm.CasadiSolver()` instead.
  self._solver = solver or self._model.default_solver


In [8]:
# plotting the results for comparison
output_variables = [
    "Negative particle surface concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "Positive particle surface concentration [mol.m-3]",
    "Negative electrode potential [V]",
    "Electrolyte potential [V]",
    "Positive electrode potential [V]",
    "Current [A]",
    "Voltage [V]",
    "Cell temperature [K]",
]

pb.dynamic_plot(
    sols,
    output_variables = output_variables,
)

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

## SEI growth models
PyBaMM options for modelling SEI growth include:
- None
- Constant: includes an SEI layer which does not grow
- Reaction limited
- Solvent-diffusion limited
- Electron-migration limited:
- Interstitial diffusion limited: Li-ion intersitial diffusion
- EC reaction limited: reaction + diffusion limited

DFN model uses graphite as anode (negative) and LiCoO3 as cathode (positive)

In [9]:
'''
Compare the results of the DFN model with various SEI growth submodels of your choice. Plot the relevant variables, and recall that you can find the list of variables to plot by typing #
model.variable_names(). What differences do you observe in the model?
'''

SEI_options = [
    "reaction limited",
    "solvent-diffusion limited",
    "electron-migration limited",
    "interstitial-diffusion limited",
    #"ec reaction limited",
]
SEI_sols = []

for option in SEI_options:
    model = pb.lithium_ion.DFN(
        name=option, 
        options={"SEI": option, "SEI porosity change": "true"}
    )    
    sim = pb.Simulation(model)
    SEI_sols.append(sim.solve([0, 180000]))
    print(option, ":", model.variable_names())

# graph plotting
output_variables = [
    "Loss of capacity to negative SEI [A.h]",
    'Negative SEI thickness [m]',
    "Voltage [V]",
    'Electrolyte potential [V]',
    "X-averaged negative electrode porosity",
    'Loss of capacity to negative SEI [A.h]',
]
pb.dynamic_plot(
    SEI_sols,
    output_variables=output_variables
)

reaction limited : ['Time [s]', 'Time [min]', 'Time [h]', 'x [m]', 'x_n [m]', 'x_s [m]', 'x_p [m]', 'r_n [m]', 'r_p [m]', 'Current variable [A]', 'Total current density [A.m-2]', 'Current [A]', 'C-rate', 'Discharge capacity [A.h]', 'Throughput capacity [A.h]', 'Discharge energy [W.h]', 'Throughput energy [W.h]', 'Negative electrode interface utilisation variable', 'X-averaged negative electrode interface utilisation variable', 'Negative electrode interface utilisation', 'X-averaged negative electrode interface utilisation', 'Positive electrode interface utilisation variable', 'X-averaged positive electrode interface utilisation variable', 'Positive electrode interface utilisation', 'X-averaged positive electrode interface utilisation', 'Negative particle crack length [m]', 'X-averaged negative particle crack length [m]', 'Negative particle cracking rate [m.s-1]', 'X-averaged negative particle cracking rate [m.s-1]', 'Positive particle crack length [m]', 'X-averaged positive particle cr

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0049064460880974, step=0.01004906446088097…

## Particle mechanics
Models for particle mechanics are also included:
- None
- Swelling only: only accounts for deformation of particles in lithiation-delithiation cycles
- Swelling and cracking: accounts for deformation + cracking during cycles

In [10]:
particle_options = [
    "swelling only",
    "swelling and cracking",
]
parameter_values = pb.ParameterValues("Ai2020")
particle_sols = []

for option in particle_options:
    model = pb.lithium_ion.DFN(
        name = option,
        options={"particle mechanics": ("swelling only", "swelling and cracking")}
    )
    sim = pb.Simulation(
        model,
        parameter_values=parameter_values)

    particle_sols.append(sim.solve([0, 3600]))

output_variables = [
    "Negative particle surface radial stress [Pa]",
    "Negative particle surface tangential stress [Pa]",
    "Negative particle surface displacement [m]",
    "Negative particle crack length [m]",
    "Positive particle surface radial stress [Pa]",
    "Positive particle surface tangential stress [Pa]",
    "Positive particle surface displacement [m]",
    "Positive particle crack length [m]",
]
pb.dynamic_plot(
    particle_sols,
    output_variables=output_variables,
)

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…